In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
# ENV["CONDA_JL_USE_MINIFORGE"] = "1"
pkgs = [
    "Revise",
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
# fasta_files = sort(filter(x -> occursin(r"\.fna$", x) && !occursin(".vcf", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
# coverages = [10, 100, 1000]

In [ ]:
# fasta_files

In [ ]:
# fasta_file = first(fasta_files)

In [ ]:
short_read_sets = unique(replace.(filter(x -> occursin(r"\.fna\.art", x), readdir(genome_dir, join=true)), r"\.[12]\.fq\.gz" => ""))

In [ ]:
threads = 2
memory = 16
for short_read_set in short_read_sets
    forward = short_read_set * ".1.fq.gz"
    reverse = short_read_set * ".2.fq.gz"
    outdir = dirname(short_read_set)
    cmd =
    """
    $(Mycelia.MAMBA) run -n trim_galore trim_galore --suppress_warn --cores $(threads) --output_dir $(outdir) --paired $(forward) $(reverse)
    """
    Mycelia.sbatch(
        job_name = "trimgalore.$(basename(short_read_set))",
        mail_user = "cameron.prybol@gmail.com",
        logdir = "$(homedir())/workspace/slurmlogs",
        partition = "batch",
        account = "mpsnyder",
        mem_gb = memory,
        cpus_per_task = threads,
        cmd = cmd)
end